## Часть 1. Обработка данных и вычисление веса

In [ ]:
import geopandas as gpd
import gdal
import matplotlib.pyplot as plt
from math import radians, sin, cos, sqrt, atan2
import shapely.wkb as wkb
import psycopg2
from geopy.distance import geodesic
from geopy.point import Point
from shapely.geometry import LineString
from qgis.core import QgsProject, QgsVectorLayer, QgsDataSourceUri, QgsFeature
from PyQt5.QtCore import QVariant
from psycopg2.extras import RealDictCursor


# открытие файла SRTM
srtm_ds = gdal.Open('C:/Users/Максим/Desktop/my.tif')

# получение геотрансформации и размерности файла
geo_transform = srtm_ds.GetGeoTransform()
x_size = srtm_ds.RasterXSize
y_size = srtm_ds.RasterYSize

# Параметры подключения к базе данных PostgreSQL
host = 'localhost'
database = 'dorogi_samara'
user = 'postgres'
password = 'Rk.rdtyysqvjhc98'

# Устанавливаем соединение с базой данных
conn = psycopg2.connect(host=host, database=database, user=user, password=password)

# Создаем курсор для выполнения запросов
cur = conn.cursor()

# Выполняем запрос для получения данных улиц из таблицы
query = "SELECT name, geom,id FROM export"
cur.execute(query)
rows = cur.fetchall()


# Создаем GeoDataFrame из полученных данных
data = gpd.GeoDataFrame(rows, columns=['name', 'geometry','id'])


# Преобразуем столбец "geom" в геометрический тип
data['geometry'] = data['geometry'].apply(lambda x: wkb.loads(x, hex=True))


# Разбиваем многочастные геометрии на отдельные части
data = data.explode()


# Добавляем координаты и название улицы в список
# функция, которая по индексу пикселя возвращает его высоту
def height_from_pixel(x, y):
    i = int((x - geo_transform[0]) / geo_transform[1])
    j = int((y - geo_transform[3]) / geo_transform[5])
    band = srtm_ds.GetRasterBand(1)
    if band.ReadAsArray(i, j, 1, 1) is not None:
        height = band.ReadAsArray(i, j, 1, 1)[0][0]
        return height
    else:
        return None
    
def calculate_distance(coord1, coord2):
    try:
        # Создание точек с координатами
        point1 = Point(coord1[0], coord1[1])
        point2 = Point(coord2[0], coord2[1])
        # Вычисление расстояния между точками с использованием модуля geopy
        dist = geodesic(point1, point2).meters
        return dist
    except TypeError as e:
        print(f"Ошибка с координатами: {coord1}, {coord2}")
        raise e


def interpolate_coordinates(coor1, coor2):
    distance = calculate_distance(coor1, coor2)
    lat1, lon1 = coor1
    lat2, lon2 = coor2
    centr = [(lat1+lat2)/2,(lon1+lon2)/2]
    if distance >= 40:
        coor_list = interpolate_coordinates(coor1, centr)
        coor_list.extend(interpolate_coordinates(centr, coor2))
        return coor_list
    else:
        return [coor1, coor2]
    
    
def difficult(dist, high):
    # Этот метод вычисляет "сложность" проезда по участку дороги, учитывая её длину и наклон.
    # `dist` содержит расстояния между последовательными точками, `high` - разницу в высоте.
    mg = 10
    A = 0
    nu = 0.06
    for i in range(len(high)):
        if high[i] >= 0:
            A += mg*(nu*dist[i]+high[i])
        else:
            A += mg*(nu*dist[i]-abs(high[i]))
    return A

# Сначала собираем все улицы и их координаты в список `all_streets`.
all_streets = []
for index, row in data.iterrows():
    coordinates = row['geometry'].coords
    street_name = row['name']
    ID = row['id']
    all_streets.append((street_name, coordinates,ID))
    
# Для каждой улицы интерполируем координаты, если расстояние между точками больше 30 метров.
for i in range(len(all_streets)):
    many_coor = []
    #if all_streets[i][0] == "Рабочая улица":
    for j in range(len(all_streets[i][1]) - 1):
        # Вычисление расстояния между двумя последовательными точками.
        distance = calculate_distance(all_streets[i][1][j], all_streets[i][1][j+1])
        # Если расстояние не превышает 30 метров, добавляем точку в список.
        if distance <= 30:
            many_coor.append(all_streets[i][1][j])
        if distance > 30:
            # Иначе интерполируем координаты между этими точками.
            new_mas = interpolate_coordinates(all_streets[i][1][j], all_streets[i][1][j+1])
            for coor in new_mas:
                many_coor.append(coor)
    many_coor.append(all_streets[i][1][len(all_streets[i][1]) - 1])
    # Обновляем список улиц новым списком координат после интерполяции.
    all_streets[i] = (all_streets[i][0], many_coor)
    

for i in range(len(all_streets)):
   # if all_streets[i][0] == "Рабочая улица":
    high_coor = []
    for j in range(0,len(all_streets[i][1])):
            height = height_from_pixel(all_streets[i][1][j][0], all_streets[i][1][j][1])
            if height is not None:
                coor = [all_streets[i][1][j],height]
                high_coor.append(coor)
    all_streets[i] = (all_streets[i][0], high_coor)  
    

    
    
# Для каждой улицы вычисляем веса в прямом и обратном направлениях, учитывая высоту.  
for i in range(len(all_streets)):
    #if all_streets[i][0] == "Рабочая улица":
    metr = []
    high = []
    for j in range(0,len(all_streets[i][1])-1):   
        distance = calculate_distance(all_streets[i][1][j][0],all_streets[i][1][j+1][0])
        metr.append(round(distance,1)) 
        high.append(all_streets[i][1][j+1][1]-all_streets[i][1][j][1])
    weight = difficult(metr,high)
    re_high = list(map(lambda x: -x, high))
    re_weight = difficult(metr,re_high)
    all_streets[i] = (all_streets[i][0], all_streets[i][1],weight,re_weight)
    
    
table_name = 'export'
new_column_name = 'weight'  
new2_column_name = 're_weight'  
#print(all_streets)


for i in range(len(all_streets)):
    value = all_streets[i][2]
    value2 = all_streets[i][3]
    query = f"UPDATE {table_name} SET {new_column_name} = {value} WHERE id_0 = '{i+1}'"
    cur.execute(query)
    query = f"UPDATE {table_name} SET {new2_column_name} = {value2} WHERE id_0 = '{i+1}'"
    cur.execute(query)
conn.commit() 
print("OK")
# Закрываем курсор и соединение
cur.close()
conn.close()

## Часть 2. Поиск оптимального пути

In [ ]:
import geopandas as gpd
import gdal
import matplotlib.pyplot as plt
from math import radians, sin, cos, sqrt, atan2
import shapely.wkb as wkb
import psycopg2
from geopy.distance import geodesic
from geopy.point import Point
from shapely.geometry import LineString
from qgis.core import QgsProject, QgsVectorLayer, QgsDataSourceUri, QgsFeature
from PyQt5.QtCore import QVariant
from psycopg2.extras import RealDictCursor


# открытие файла SRTM
srtm_ds = gdal.Open('C:/Users/Максим/Desktop/my.tif')

# получение геотрансформации и размерности файла
geo_transform = srtm_ds.GetGeoTransform()
x_size = srtm_ds.RasterXSize
y_size = srtm_ds.RasterYSize

# Параметры подключения к базе данных PostgreSQL
host = 'localhost'
database = 'dorogi_samara'
user = 'postgres'
password = 'Rk.rdtyysqvjhc98'

# Устанавливаем соединение с базой данных
conn = psycopg2.connect(host=host, database=database, user=user, password=password)

# Создаем курсор для выполнения запросов
cur = conn.cursor()






# Функция для получения ближайшей вершины к заданным координатам
def get_nearest_vertex(lat, lon, conn):
    # Создаем курсор для выполнения операций с базой данных
    cur = conn.cursor()
    
    # Формируем SQL-запрос для поиска ближайшей вершины
    query = f"""
    SELECT id, the_geom, ST_Distance(the_geom, ST_SetSRID(ST_MakePoint({lon}, {lat}), 4326)) AS distance
    FROM export_vertices_pgr
    ORDER BY distance
    LIMIT 1;
    """
    
    # Выполняем запрос и получаем результаты
    cur.execute(query)
    
    # Получаем одну запись
    row = cur.fetchone()
    
    # Закрываем курсор
    cur.close()
    
    # Возвращаем id ближайшей вершины
    if row:
        return row[0]
    else:
        return None

# Функция для использования pgRouting для поиска маршрута
def find_route(start_vertex, end_vertex, conn):
    with conn.cursor(cursor_factory=RealDictCursor) as cur:
        # Обратите внимание на использование "weight AS cost" в SQL-запросе.
        cur.execute("""
            SELECT * FROM pgr_dijkstra(
                'SELECT id_0 AS id, source, target, weight AS cost, re_weight AS reverse_cost FROM export',
                %s, %s, directed := true
            )
        """, (start_vertex, end_vertex))
        route = cur.fetchall()
        print(route)
        return route

# Функция для визуализации маршрута в QGIS
def visualize_route(route_segments, conn):
    #print(f"Визуализация маршрута с сегментами: {route_segments}") # Добавляем отладочный вывод
    route_layer = QgsVectorLayer('LineString', 'route', 'memory')
    pr = route_layer.dataProvider()
    pr.addAttributes([QgsField("id", QVariant.Int)])
    route_layer.updateFields()
    ids = [segment['edge'] for segment in route_segments if segment['edge'] > 0]

    for edge_id in ids:
        #print(f"Добавление сегмента с ID: {edge_id}")
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute("""
                SELECT ST_AsText(geom) as geom FROM export WHERE id_0 = %s
            """, (edge_id,))
            edge = cur.fetchone()
            if edge:
                feat = QgsFeature()
                feat.setGeometry(QgsGeometry.fromWkt(edge['geom']))
                feat.setAttributes([edge_id])
                pr.addFeature(feat)

    QgsProject.instance().addMapLayer(route_layer)


# Начальная и конечная точки (широта и долгота)

start_point = (53.195088,50.093820)  

end_point = (53.190310,50.113497)    



# Получаем ближайшие вершины к заданным точкам

start_vertex = get_nearest_vertex(start_point[0], start_point[1], conn)
print(f"Ближайшая вершина к начальной точке: {start_vertex}")

end_vertex = get_nearest_vertex(end_point[0], end_point[1], conn)
print(f"Ближайшая вершина к конечной точке: {end_vertex}")


# Ищем маршрут между вершинами

route = find_route(start_vertex, end_vertex, conn)
print(f"Найденный маршрут: {route}")


# Визуализируем маршрут в QGIS

visualize_route(route, conn)

def calculate_travel_time(route, energy_per_unit):
    """
    Вычисляет время в пути на основе маршрута и энергетической стоимости.

    Параметры:
    route (list): список сегментов маршрута, где каждый сегмент содержит 'cost' как энергетическую стоимость
    energy_per_unit (float): количество энергии, которое в среднем тратится на единицу работы

    Возвращает:
    float: общее время в пути
    """
    total_energy = sum(segment['cost'] for segment in route)
    # Предположим, что энергетическая стоимость может быть переведена во время
    # путем деления на количество энергии, затрачиваемой за единицу времени
    return total_energy / energy_per_unit

def calculate_total_work(route):
    """
    Вычисляет общую энергетическую стоимость маршрута.

    Параметры:
    route (list): список сегментов маршрута, где каждый сегмент содержит 'cost'

    Возвращает:
    float: общая энергетическая стоимость
    """
    return sum(segment['cost'] for segment in route)

# Пример использования функций:
energy_per_unit = 1  # Это значение необходимо подобрать или вычислить
travel_time = calculate_travel_time(route, energy_per_unit)
print(f"Время в пути: {travel_time} единиц времени")

# Расчет общей энергетической стоимости маршрута
total_work = calculate_total_work(route)
print(f"Общая энергетическая стоимость маршрута: {total_work} единиц работы")




# Закрываем курсор и соединение
cur.close()
conn.close()